In [1]:
#%cd 'Desktop/Projects/energy-consumption-forecaster'

In [2]:
from datetime import datetime
from os import listdir

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import pyarrow
import seaborn
import sys
import time

sys.path.append('../tools/')
from utils import *

In [3]:
total = pd.read_parquet('../data/enertalk-dataset/00/20161101/00_total.parquet.gzip')
wash = pd.read_parquet('../data/enertalk-dataset/00/20161101/02_washing-machine.parquet.gzip')

#total['datetime'] = total['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.))
#wash['datetime'] = wash['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.))

wash.head()

,active_power,reactive_power,timestamp
0,0.12,6.12,1.477959e+12
1,0.00,5.96,1.477959e+12
2,0.05,6.06,1.477959e+12
3,0.05,5.98,1.477959e+12
4,0.02,6.05,1.477959e+12


In [4]:
def load_data():
    dates = listdir('../data/enertalk-dataset/00')
    start = time.time()
    
    for date in dates:
        
        if date != ".DS_Store":
            print(f"Reading file {date} | File {dates.index(date)+1}/{len(dates)}")
            total = pd.read_parquet('../data/enertalk-dataset/00/'+date+'/00_total.parquet.gzip')
            if dates.index(date) == 0:
                raw = pd.DataFrame(columns=total.columns)
            else:
                raw = pd.concat([total, raw])
    end = time.time()
    print(f"Total time elapsed {end-start} seconds, or {(end-start)/60} minutes.")
    return raw

In [5]:
raw = load_data()

Reading file 20170113 | File 1/87
Reading file 20170114 | File 2/87
Reading file 20170122 | File 3/87
Reading file 20170125 | File 4/87
Reading file 20161209 | File 5/87
Reading file 20161207 | File 6/87
Reading file 20170124 | File 7/87
Reading file 20170123 | File 8/87
Reading file 20170115 | File 9/87
Reading file 20170112 | File 10/87
Reading file 20161206 | File 11/87
Reading file 20161201 | File 12/87
Reading file 20161208 | File 13/87
Reading file 20161230 | File 14/87
Reading file 20161112 | File 15/87
Reading file 20161115 | File 16/87
Reading file 20161123 | File 17/87
Reading file 20161125 | File 18/87
Reading file 20161122 | File 19/87
Reading file 20161114 | File 20/87
Reading file 20161113 | File 21/87
Reading file 20161109 | File 22/87
Reading file 20161107 | File 23/87
Reading file 20161106 | File 24/87
Reading file 20161101 | File 25/87
Reading file 20161130 | File 26/87
Reading file 20161212 | File 27/87
Reading file 20161215 | File 28/87
Reading file 20161223 | File 

In [6]:
raw.shape

(109399395, 3)

In [7]:
def clean_data(
    df,
    app=False,
    resample='15S'
):
    start = time.time()
    df['timestamp'] = df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.))
    df['timestamp'] = df['timestamp'].apply(lambda x: x.replace(microsecond=0)).dt.floor(resample)
    df = df.groupby('timestamp').mean()
    df = df.reset_index()
    if app == True:
        df['on'] = (df['active_power'] > 15).astype(int)
    end = time.time()
    time_elapsed = end - start
    print(f"Time elapsed: {str(datetime.timedelta(seconds=time_elapsed))}")
    return df

In [8]:
raw = clean_data(raw)

Time elapsed: 0:17:13.516626


In [12]:
raw.to_csv(
    "../data/intermediate/cleaned.csv",
    index=False
)

In [13]:
pd.read_csv("../data/intermediate/cleaned.csv")

,timestamp,active_power,reactive_power
0,2016-10-31 17:00:00,189.952650,-61.204872
1,2016-10-31 17:00:15,157.639905,-79.938286
2,2016-10-31 17:00:30,155.861315,-79.832582
3,2016-10-31 17:00:45,163.025980,-79.870882
4,2016-10-31 17:01:00,162.825952,-79.833690
...,...,...,...
494656,2017-01-31 15:58:45,549.240711,-143.336447
494657,2017-01-31 15:59:00,556.102386,-144.161818
494658,2017-01-31 15:59:15,562.742972,-144.952028
494659,2017-01-31 15:59:30,561.509886,-145.039091
